In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import time

from prune import *

In [3]:
#TODO
#add new network

In [4]:
batch_size_train = 64
batch_size_test = 1000

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.smx = nn.LogSoftmax(dim=1)

    def forward(self, x):      
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.smx(x)
        return x

net = Net()
#net

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.smx = nn.LogSoftmax(dim=1)

    def forward(self, x):      
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.smx(x)
        return x

net = Net()
#net

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [8]:
#initialize locked masks:
locked_masks = {n: torch.zeros(w.size(), dtype=torch.bool) for n, w in net.named_parameters()}

In [9]:
#start time
start=time.time()

for prune_step in range(5):
    
    if prune_step > 0: 
        print('Start Pruning')
        prune(net,locked_masks, prune_random=False, prune_weight=True, prune_bias=False, ratio=0.75,
          threshold=None, threshold_bias=None, function=None, function_bias=None, prune_across_layers=True) 
        print('Done Pruning')
        
        correct(test_loader,net)
    
    prune_diag(net,locked_masks)
    
    #print('prune diag time: ',time.time()-s2)
    
    for epoch in range(4):  # loop over the dataset multiple times
        
        running_loss = 0.0
        for i, data in enumerate(train_loader,0):
            
                
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.view(inputs.shape[0], -1)
                       
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + prune_grad + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            net = prune_grad(net,locked_masks) #zeros gradients of the pruned weights
            optimizer.step()
            

            # print statistics
            running_loss += loss.item()
            if i % 200 == 199:    # print every 200 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 200))
                running_loss = 0.0
            

        correct(test_loader,net)
        
        
    print('Finished Training')

print('time: ',time.time()-start)

name       pruned percentage
fc1.weight 0.0
fc1.bias 0.0
fc2.weight 0.0
fc2.bias 0.0
[1,   200] loss: 1.246


KeyboardInterrupt: 

In [10]:
for n, w in net.named_parameters():
    #print(len(w.data.numpy().shape))
    print(len(w.shape))
    print(w)

2
Parameter containing:
tensor([[-0.0242, -0.0143,  0.0246,  ..., -0.0132,  0.0103, -0.0080],
        [-0.0064, -0.0215,  0.0081,  ...,  0.0096,  0.0123,  0.0161],
        [-0.0226, -0.0011,  0.0081,  ...,  0.0302, -0.0009,  0.0268],
        ...,
        [-0.0290, -0.0224,  0.0158,  ..., -0.0099,  0.0239,  0.0346],
        [-0.0084, -0.0130, -0.0205,  ...,  0.0153, -0.0063,  0.0184],
        [ 0.0089,  0.0247,  0.0075,  ...,  0.0138, -0.0371,  0.0335]],
       requires_grad=True)
1
Parameter containing:
tensor([-1.8727e-02,  3.4951e-02,  1.7086e-02, -2.7551e-02, -1.5423e-02,
        -1.0521e-02, -2.8784e-02,  1.9701e-02,  1.1328e-02,  4.6458e-03,
         3.0008e-02,  3.2366e-02, -1.2112e-02, -1.7444e-03, -6.7797e-03,
         8.5361e-03, -2.0696e-02,  2.6563e-03,  2.5735e-02, -1.3409e-02,
         2.1776e-02,  1.3081e-02,  1.0480e-02,  2.5586e-02,  5.7624e-03,
        -2.5663e-02,  2.5561e-02,  1.8951e-02,  3.8471e-02,  1.3479e-02,
        -2.5401e-02,  1.5525e-02,  1.9481e-02,  1.925